# EnTK Seals Pipeline notebook.

This notebook provides a prototypical implementation of the Seal use case, as it is shown in the [Seal Execution Model](https://docs.google.com/document/d/1E79LfwXG1ZJ1fTQiGsDvSggE6BJSDEqAyHKcCxjqgoY/edit?ts=5af5d13e). Each cell of the notebook creates a necesary component of the pipeline definition.

In [1]:
from radical.entk import Pipeline, Stage, Task, AppManager, ResourceManager

## Pipeline Definition

The next cell defines the prototype pipeline for the Seal Use case. The pipeline has two stages, and each stage has a single task.
The first stage executes the prediction and the second the detection.

What needs to be added is [Stage number 0](https://docs.google.com/document/d/1E79LfwXG1ZJ1fTQiGsDvSggE6BJSDEqAyHKcCxjqgoY/edit?ts=5af5d13e) and the last stage that aggregates the results. Also, the single task in both cases should be broken to multiple tasks based on the number of images.

In [ ]:
def generate_pipeline(name, stages):  #generate the pipeline of prediction and blob detection

    # Create a Pipeline object
    p = Pipeline()
    p.name = name

    for s_cnt in range(stages):

        # Create a Stage object
        s = Stage()
        s.name = 'Stage %s'%s_cnt
        if(stage==1):
            # Create Task 1, training
            t1 = Task()
            t1.name = 'Predictor'
            t1.pre_exec = ['module load python3',
                          'module load cuda',
                          'module load opencv',
                          'module load gcc',
                          'mkdir -p classified_images/crabeater',
                          'mkdir -p classified_images/weddel',
                          'mkdir -p classified_images/packi-ice',
                          'mkdir -p classified_images/other'
                         ]
            t1.executable = 'python3'   # Assign executable to the task   
            # Assign arguments for the task executable
            t1.arguments = ['pt_predict.py','-class_names','crabeater','weddel','pack-ice','other']
            s.add_tasks(t1)
        else:
            # Create Task 2, 
            t2 = Task()
            t2.pre_exec = ['module load python3',
                          'module load cuda',
                          'module load opencv',
                          'module load gcc',
                          'mkdir -p classified_images/crabeater',
                          'mkdir -p classified_images/weddel',
                          'mkdir -p classified_images/packi-ice',
                          'mkdir -p classified_images/other'
                         ]
            t2.name = 'Blob_detector'         
            t2.executable = ['python3']   # Assign executable to the task   
            # Assign arguments for the task executable
            t2.arguments = ['blob_detector.py']
            t2.link_input_data = ['$Pipline_%s_Stage_%s_Task_%s/classified_images/crabeater/*'%(p.uid, s.uid, t1.uid),
                                  '$Pipline_%s_Stage_%s_Task_%s/classified_images/weddel/*'%(p.uid, s.uid, t1.uid),
                                  '$Pipline_%s_Stage_%s_Task_%s/classified_images/packi-ice/*'%(p.uid, s.uid, t1.uid),
                                  '$Pipline_%s_Stage_%s_Task_%s/classified_images/other/*'%(p.uid, s.uid, t1.uid)]
            t2.download_output_data = ['classified_images'] #Download resuting images 
            s.add_tasks(t2)

    # Add Stage to the Pipeline
    p.add_stages(s)

    return p

In [ ]:
p1 = generate_pipeline(name='Pipeline 1', stages=2)

In [ ]:
 res_dict = {'resource': 'xsede.bridges',
             'username': '' #username is not currently included in ensemble toolkit,               so an error arises
             'walltime': 10,
             'cores': 1,
             'project': '',
    }

In [ ]:
# Create Application Manager
appman = AppManager(port=32773)

# Assign resource manager to the Application Manager
appman.resource_manager = rman

# Assign the workflow as a set of Pipelines to the Application Manager
appman.assign_workflow(set([p]))

# Run the Application Manager
appman.run()